# 1, Import libraries

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from bs4 import BeautifulSoup
import requests
import geocoder
from sklearn.cluster import KMeans

# 2, Import data from previous notebook

In [4]:
df = pd.read_csv("file2.csv", index_col = 0)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 3, Explore Toronto

In [14]:
tor_df = df[df.Borough.str.contains("Toronto")].reset_index().drop(columns="index")
tor_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
tor_map = folium.Map(location=[tor_df.Latitude.mean(), tor_df.Longitude.mean()], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tor_map)  
    
tor_map

In [17]:
CLIENT_ID = '1LH5UHA0XLZSSFNYESGAVK5TTPIJNG1VACOREOXGAXEMRGIE' # your Foursquare ID
CLIENT_SECRET = 'Z1PHKBXIOGYDCISVKKVJWGSTBIK2KL0YFVJLWWRRBGQPSZPO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
tor_venues = getNearbyVenues(names=tor_df['Neighbourhood'],
                                   latitudes=tor_df['Latitude'],
                                   longitudes=tor_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [22]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [23]:
tor_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",41,41,41,41,41,41
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",84,84,84,84,84,84
Christie,18,18,18,18,18,18


# Analyze the neibourhood

In [37]:
tor_onehot = pd.get_dummies(tor_venues[["Venue Category"]], prefix = "", prefix_sep = "")
tor_onehot["Neighborhood"] = tor_venues["Neighborhood"]
tor_cols = list(tor_onehot.columns)
tor_cols.remove("Neighborhood")
tor_cols = ["Neighborhood"] + tor_cols
tor_onehot = tor_onehot[tor_cols]



In [38]:
tor_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
tor_group = tor_onehot.groupby("Neighborhood").mean().reset_index()
tor_group

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.01,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.018519,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.043478
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.055556
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011765,0.000000,...,0.000000,0.00000,0.00,0.000000,0.011765,0.000000,0.000000,0.011765,0.00,0.011765
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.047619,0.000000,0.071429,0.011905,0.00,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.012195,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012195,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.012195


In [85]:
num_top_venues = 10
list_cols = ["Neighborhood", "1st", "2nd", "3rd", "4th", "5th", "6th", "7th", "8th", "9th", "10th"]
tor_top10 = pd.DataFrame(columns = list_cols)
for i in tor_group.index:
    temp_nei = tor_group.loc[i, "Neighborhood"]
    temp_df = pd.DataFrame(tor_group.loc[i, :][1:]).reset_index()
    temp_df.columns = ["Venue", "Freq"]
    temp_df["Freq"] = round(temp_df.Freq.astype(float), 2)
    temp_df = temp_df.sort_values("Freq", ascending = False).head(num_top_venues).reset_index(drop = True)
    tor_top10.loc[i, :] = np.append([temp_nei], list(temp_df.Venue.values))
    print("---{}---".format(temp_nei))
    print(temp_df)
    

---Adelaide, King, Richmond---
              Venue  Freq
0       Coffee Shop  0.07
1              Café  0.04
2        Steakhouse  0.04
3               Bar  0.04
4   Thai Restaurant  0.03
5      Burger Joint  0.03
6    Cosmetics Shop  0.03
7            Bakery  0.03
8    Breakfast Spot  0.03
9  Asian Restaurant  0.03
---Berczy Park---
                Venue  Freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.06
2            Beer Bar  0.04
3                Café  0.04
4         Cheese Shop  0.04
5              Bakery  0.04
6  Seafood Restaurant  0.04
7          Steakhouse  0.04
8      Farmers Market  0.04
9        Concert Hall  0.02
---Brockton, Exhibition Place, Parkdale Village---
                    Venue  Freq
0          Breakfast Spot  0.09
1                    Café  0.09
2               Nightclub  0.09
3             Coffee Shop  0.09
4           Grocery Store  0.04
5  Furniture / Home Store  0.04
6           Burrito Place  0.04
7            Climbing Gym  0.04
8       Convenienc

In [87]:
tor_top10.head()

,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Burger Joint,Cosmetics Shop,Bakery,Breakfast Spot,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Café,Cheese Shop,Bakery,Seafood Restaurant,Steakhouse,Farmers Market,Concert Hall
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Nightclub,Coffee Shop,Grocery Store,Furniture / Home Store,Burrito Place,Climbing Gym,Convenience Store,Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Burrito Place,Spa,Fast Food Restaurant,Butcher,Farmers Market,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Airport Food Court,Airport Gate,Harbor / Marina,Bar,Airport,Sculpture Garden


In [88]:
k = 5
tor_group_clus = tor_group.drop(columns = ["Neighborhood"])
kmeans = KMeans(n_clusters = k, random_state = 0).fit(tor_group_clus)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       4, 0, 1, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# 4, Create output file for future usage

In [38]:
df1

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [39]:
df1.to_csv("file3.csv")